#### Desde el log de conversaciones de CAD obtiene a través de GenAI el tema, problema, si fue resuelto o no y cómo lo resolvió el CAD

In [93]:
import warnings
warnings.filterwarnings('ignore')

In [94]:
archivo_csv_importar = 'data/bot_asesor_todas.csv'
archivo_csv_exportar = 'data/bot_asesor_todas_genai.csv'
titulo_columna_analizar_cad = 'cuerpo_mensaje_tx'
id_columna = 'conversacion_id'
columna_nueva_1 = 'tema'
columna_nueva_2 = 'problema'
columna_nueva_3 = 'resuelto'
columna_nueva_4 = 'como_resolvio'
grabar_archivo = True


In [95]:
# importamos librerías
import numpy as np
import pandas as pd
import json
import requests
import re
import time
import random
import json


### Importar Datasets

In [96]:
#importo datasets
df_cluster = pd.read_csv(archivo_csv_importar, encoding='utf8')
df_cad = pd.read_csv('cad.csv', encoding='utf8')

In [97]:
df_cluster.head(5)

,conversacion_id,texto_preprocesado,new_cluster
0,64e510a1-eb7b-4b27-bd26-19f88fb4ce22,necesito hablar asesor,x
1,c68e4cee-92bc-4868-ad78-dd8b9fa86389,podría hablar representante preguntaron número...,4
2,82cfc9f4-ffcb-49d2-ad9d-bb6a4fe70cb4,ayuda asistente,x
3,56c88f71-7a1e-49e6-9070-62a682699f53,necesito hablar hablar alguien,x
4,078d23c6-e80f-43f6-a93f-45a920e86db8,hablar representante,x


In [98]:
df_cluster.shape

(307, 3)

In [99]:
df_cad.head(5)

,caso_id,mensaje_id,indice_cd,fecha_ts,cuerpo_mensaje_tx,actor,conversacion_id
0,13467703,69222608,2,2024-09-02 16:59:09,42307388 - #CLIENTE VALIDADO#,Cliente,2be02be1-d520-4bbd-ba49-a303cd4603a4
1,13467703,69222676,3,2024-09-02 16:59:36,Dale,Cliente,2be02be1-d520-4bbd-ba49-a303cd4603a4
2,13467703,69223054,4,2024-09-02 17:02:25,Hola Cristian. ¿Cómo estás? Mi nombre es Milen...,CAD,2be02be1-d520-4bbd-ba49-a303cd4603a4
3,13467703,69223057,5,2024-09-02 17:02:25,Veo que tenés inconvenientes con tus claves de...,CAD,2be02be1-d520-4bbd-ba49-a303cd4603a4
4,13467703,69223113,6,2024-09-02 17:02:55,Hola Milena,Cliente,2be02be1-d520-4bbd-ba49-a303cd4603a4


In [100]:
df_cad.shape

(91436, 7)

In [101]:
# Realizar el join entre los DataFrames basado 'conversacion_id'
df_join = pd.merge(df_cluster, df_cad, left_on=id_columna, right_on=id_columna, how='inner')

In [102]:
df_join.head(5)

,conversacion_id,texto_preprocesado,new_cluster,caso_id,mensaje_id,indice_cd,fecha_ts,cuerpo_mensaje_tx,actor
0,64e510a1-eb7b-4b27-bd26-19f88fb4ce22,necesito hablar asesor,x,13474343,69161592,1,2024-09-02 09:45:26,30977646 - #CLIENTE VALIDADO#,Cliente
1,64e510a1-eb7b-4b27-bd26-19f88fb4ce22,necesito hablar asesor,x,13474343,69162439,2,2024-09-02 09:52:40,¡Buenos Días Sabrina! ¿Cómo estás? Mi nombre e...,CAD
2,64e510a1-eb7b-4b27-bd26-19f88fb4ce22,necesito hablar asesor,x,13474343,69162440,3,2024-09-02 09:52:41,Sabri coméntame ¿en qué te puedo ayudar?,CAD
3,64e510a1-eb7b-4b27-bd26-19f88fb4ce22,necesito hablar asesor,x,13474343,69162635,4,2024-09-02 09:54:33,Hola Mateo buen día.,Cliente
4,64e510a1-eb7b-4b27-bd26-19f88fb4ce22,necesito hablar asesor,x,13474343,69162674,5,2024-09-02 09:54:55,Quería saber cual es mi saldo anterior a pagar,Cliente


In [103]:
df_join.shape

(4874, 9)

In [104]:
df_join = df_join.drop(df_join[df_join['cuerpo_mensaje_tx']=='--Vacío--'].index)

### GENAI 

In [105]:
## GENAI ##
model = "mistralai/mixtral-8x7b-instruct-v01"
api_key = "n5MIR-sK-VCa03SZD1QK-SHMoSudruiX9SozoLekshcn" #os.getenv("API_KEY", None)
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com/" #os.getenv("IBM_CLOUD_URL", None)
project_id = "84c453f6-00ab-4f67-afca-061aec4c0bbc" #os.getenv("PROJECT_ID", None)
url_cloud = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"
creds = {
    "url": ibm_cloud_url,
    "apikey": api_key 
}
token = ''
# propiedades
max_tokens_limit = 8192
max_tokens = 4096
temperature = 0.7
top_p = 1.0
random_seed = random.randrange(1,99)
total_tokens_used = 0
response_seconds = 0.0
total_response_seconds = 0.0
avg_response_seconds = 0.0

    

In [106]:
def generateTokenIBM(myApik):    
    #curl -X POST 'https://iam.cloud.ibm.com/identity/token' 
    url = "https://iam.cloud.ibm.com/identity/token"
    # -d 'grant_type=urn:ibm:params:oauth:grant-type:apikey & apikey=${myApik}
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": myApik
    }
    # -H 'Content-Type: application/x-www-form-urlencoded'
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    #curl -X POST 'https://iam.cloud.ibm.com/identity/token' -H 'Content-Type: application/x-www-form-urlencoded' 
    # -d 'grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey=${myApik}'
    response = requests.post(url, data=data, headers=headers)  ### La request de CURL.
    access_token = None
    # Verifica si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Parsea la respuesta JSON
        #print("response: ",response,"\n\n")
        json_data = json.loads(response.text)
        # Extrae el valor de 'access_token' de la respuesta JSON
        access_token = json_data.get('access_token')
    else:
        print(f"La solicitud falló con el código de estado {response.status_code}.")
        print(f"Response.text: ")
        json_data = json.loads(response.text)
        print(json_data)
    return access_token

In [107]:
def clean_text(text):
    """
    text: a string
    return: modified initial string
    """
    # Lets do some text cleanup
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    REMOVE_NUM = re.compile('[\d+]')
    
    text = str(text)
    # lowercase text
    text = text.lower() 
    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    # Remove the XXXX values
    text = text.replace('x', '') 
    # Remove white space
    text = REMOVE_NUM.sub('', text)
    #  delete symbols which are in BAD_SYMBOLS_RE from text
    #text = BAD_SYMBOLS_RE.sub('', text) 
    # delete stopwords from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    # removes any words composed of less than 2 or more than 21 letters
    text = ' '.join(word for word in text.split() if (len(word) >= 2 and len(word) <= 21))
    # Stemming the words
    #text = ' '.join([stemmer.stem(word) for word in text.split()])
    # Stemming the words
    if forma_tokenizar == 's':
        text = ' '.join([stemmer.stem(str(word)) for word in text.split()])
    elif forma_tokenizar == 'l': #lemmatization
        text = ' '.join(x.lemma_ for x in nlp(text))
    return text

# Limpia texto de caracteres extendidos y espacios en blanco
def clean_text_message(texto):
    texto_str = str(texto)
    # Definir un patrón que permita caracteres ASCII imprimibles y caracteres extendidos del español
    caracteres_permitidos = re.compile(r'[^\x20-\x7E\u00A1-\u00FF]')
    texto_limpio = caracteres_permitidos.sub('', texto_str)
    # Eliminar espacios en blanco innecesarios al principio y al final del texto
    texto_limpio = texto_limpio.strip()
    # Reemplazar más de dos espacios en blanco consecutivos por un solo espacio
    texto_limpio = re.sub(r'\s{2,}', ' ', texto_limpio)
    # Reemplaza basura de json
    return texto_limpio

# extrae la pregunta
def extraer_entre_caracteres(cadena, caracter_inicio, caracter_fin):
    try:
        inicio = cadena.index(caracter_inicio)
        fin = cadena.index(caracter_fin, inicio) + 1
        return cadena[inicio:fin]
    except ValueError:
        return cadena

# extrae texto json con [] o con {}
def extract_text_between_characters(input_string, characters=""):
    # Utiliza una expresión regular para encontrar todo el texto entre "{...}"
    if characters=="{}":
        pattern = re.compile(r'\{(.*?)\}')
        matches = pattern.findall(input_string)
    elif characters=="[]":
        pattern = re.compile(r'\[(.*?)\]')
        matches = pattern.findall(input_string)
    elif characters=="¿?":
        pattern = re.compile(r'\¿(.*?)\?')
        matches = pattern.findall(input_string)
    else:
        matches = input_string
    return str(matches)

# convierte una lista a json string
def list_to_json(json_lst):
    json_str = json.dumps(json_lst, ensure_ascii=False) if type(json_lst) == list else json_lst
    json_str = clean_text(json_str)
    return json_str

# convierte un json string a lista
def json_to_list(json_str, characters=""):
    #json_str = clean_text_message(json_str)
    decoded_json_str = json_str.encode().decode('utf-8-sig')
    #decoded_json_str = clean_text(decoded_json_str)
    decoded_json_str = decoded_json_str.replace("```json", "")
    decoded_json_str = decoded_json_str.replace("```", "")
    decoded_json_str = decoded_json_str.replace("}[{", "[{")
    decoded_json_str = decoded_json_str.replace("['{", "[{")
    decoded_json_str = decoded_json_str.replace("}']", "}]")
    #if characters=="":
    #    decoded_json_str = extract_text_between_characters(decoded_json_str, characters)
    #print(decoded_json_str)
    try:
        json_lst = json.loads(decoded_json_str)
    except:
        json_lst = decoded_json_str
    return json_lst

# convierte un json string a dataframe
def json_to_dataframe(json_str):
    try:
        dict = json.loads(json_str)
    except:
        dict = json_str
    # converting json dataset from dictionary to dataframe
    #train = pd.DataFrame.from_dict(dict, orient='columns')
    train = pd.DataFrame(dict)
    train.reset_index(level=0, inplace=True)
    #train = train.style.hide()
    return train


In [108]:
def chat_with_model(prompt_txt, model, temperature, max_tokens=250):

    start_time = time.time()
    
    if model == "mistralai/mixtral-8x7b-instruct-v01" or model == "mistralai/mistral-large":
        prompt = prompt_txt #mixtral_array2string(messages) # convierte array de prompts a string mixtral
        url = url_cloud
        headers = {
        	"Accept": "application/json",
        	"Content-Type": "application/json",
        	"Authorization": "Bearer " + token
        }
        data = {
            "model_id": model,
            "project_id": project_id,
            #"messages": messages, 
            # en el input debe pasarse un string con todo el prompt, ej: <s>[INST] bla bla [INST]</s>
            # sino tira este error: input must be of type string
            #"input": f"""<s>[INST] {prompt_system} [INST]</s> 
            #[INST] {tmp_prompt} [INST]</s> """, 
            "input": prompt,
            "parameters": {
                "decoding_method": "sample", "temperature": temperature, "top_p": top_p, "top_k": 50, "typical_p": 1
                #"decoding_method": "greedy"
                ,"min_new_tokens": 1
                ,"max_new_tokens": max_tokens
                ,"random_seed": random_seed
                ,"stop_sequences": ["?"]
                ,"repetition_penalty": 1.1
            },
            "moderations": {}
        }
        # ejecuta la api
        response = requests.post(url, headers=headers, json=data)
        if response.status_code != 200:
            try:
                raise Exception("Non-200 response: " + str(response.text))
            except Exception as Argument:
                generated_text = "ERROR: " + str(Argument)
                total_tokens = 0
                response_time = 0.0
        else: # == 200
            response_data = response.json()
            generated_text = response_data['results'][0]['generated_text']
            total_tokens = str(int(response_data['results'][0]['input_token_count']) + int(response_data['results'][0]['generated_token_count']))
    
    else: # gpt
        generated_text = "error de modelo llm elegido"
        total_tokens = 0

    end_time = time.time()
    # mide tiempo de respuesta
    response_time = end_time - start_time
    
    return generated_text, int(total_tokens), round(response_time, 2)


In [109]:
# Función para interactuar con la API de ChatGPT
def ejecutar_modelo(str_param):
    
    prompt = f"""<s>[INST] Dado el siguiente diálogo de chat en formato JSON entre un cliente y un asesor de atención al cliente (llamado CAD) de un banco: 
    {str_param} 
    Sigue estrictamente las siguientes instrucciones:
    1. Responde los siguientes puntos 2, 3, y 4 únicamente en formato JSON de esta manera y un solo registro nada más: [{{"tema":"<respuesta del siguiente punto 2>", "problema":"<respuesta del siguiente punto 3>", "resuelto":"<respuesta del siguiente punto 4>", "como_resolvio":"<respuesta del siguiente punto 5>"}}] 
    2. Responde de qué trata el tema en menos de 10 palabras y en español latinoamericano únicamente. No agregues comentarios ni caracteres extendidos. 
    3. Responde cuál fue el problema del cliente en menos de 10 palabras y en español latinoamericano únicamente. No agregues comentarios ni caracteres extendidos. 
    4. Responde: "SI" (si el asesor resolvió la inquietud del cliente), "NO" (el asesor no pudo resolver la inquietud del cliente), "NO SE" (si no clarifica la resolución de la inquietud). 
    5. Si tu respuesta del punto anterior fue "SI" entonces responde brevemente cómo el asesor lo resolvió en menos de 12 palabras. Si tu respuesta del punto anterior fue "NO" o "NO SE", entonces responde solamente "-". 
    6. Si has encontrado más de un tema en la conversación, solamente selecciona un solo tema, el más importante y respetando el formato JSON anterior. No agregues comentarios. 
     [/INST]"""
    
    response, total_tokens_used, response_seconds = chat_with_model(prompt, model, temperature, max_tokens)

    return response, total_tokens_used, response_seconds


In [110]:
# Función principal para iterar sobre las conversaciones y generar el DataFrame
#def proceso_genai_df(df):

#    genai = []
    # Iterar sobre el DataFrame
#    for index, row in df.iterrows():
#        if gramas == 0:
#            str_gramas = f"Bigramas: {str(row['bigrams'])} y Trigramas: {str(row['trigrams'])} "
            # Enviar la conversación a la API y obtener la respuesta
#            response, total_tokens_used, response_seconds = ejecutar_modelo(str_gramas, gramas)
#        else: # 2, 3
            # Enviar la conversación a la API y obtener la respuesta
#            response, total_tokens_used, response_seconds = ejecutar_modelo(str(row[titulo_columna_analizar]), gramas)
        # reemplazar comas por nada
#        response =  response.replace(',', '') 
#        if gramas == 0:
#            genai.append({"cluster": row["cluster"], "bigrams": row["bigrams"], "trigrams": row["trigrams"], titulo_columna_nueva: response, "tokens": total_tokens_used, "seconds": response_seconds})
#        else: # 2, 3
#            genai.append({"cluster": row["cluster"], "grams": row["grams"], titulo_columna_nueva: response, "tokens": total_tokens_used, "seconds": response_seconds})
    # agrega columna nueva al dataframe con los valores generados
#    df_genai = pd.DataFrame(genai)

#    return df_genai

In [111]:
# Función principal para iterar sobre las conversaciones y generar el DataFrame
def proceso_genai(df):

    global token
    #genera token
    token = generateTokenIBM(api_key)
    
    j = 0
    genai = []
    start_time = time.time()
    
    # Iterar sobre cada conversacion_id único
    for conversacion_id in df['conversacion_id'].unique():
        # Filtrar las filas para esta conversacion_id
        conversacion_df = df[df['conversacion_id'] == conversacion_id]
        # Iterar sobre las filas de la conversación
        for i in range(1, len(conversacion_df)):
            fila_actual = conversacion_df.iloc[i]
            fila_anterior = conversacion_df.iloc[i - 1]
            #1ra utterance
            if i==1:
                dialogo_data = [] # Crear una lista para almacenar los registros del DataFrame dialogo
            dialogo_data.append({'conversacion_id': fila_anterior['conversacion_id'], 'actor': fila_anterior['actor'], 'dialogo': clean_text_message(fila_anterior['cuerpo_mensaje_tx'])})
        # Crear el DataFrame dialogo
        df_dialogo = pd.DataFrame(dialogo_data)
        df_dialogo_json = df_dialogo.to_json(orient="records", force_ascii=False)
        # Enviar la conversación a la API y obtener la respuesta
        response, total_tokens_used, response_seconds = ejecutar_modelo(df_dialogo_json)
        # reemplazar comas por nada
        #response =  response.replace(',', '') 
        #response = extract_text_between_characters(response, "[]")
        #print('\n'+response)
        #df_response = json_to_dataframe(response) # convierte json a dataframe (no funciona bien)
        df_response = json_to_list(response) # convierte json a lista
        
        try:
            # apendeamos resultado
            genai.append({'conversacion_id': fila_actual['conversacion_id'], columna_nueva_1: df_response[0]["tema"], columna_nueva_2: df_response[0]["problema"], columna_nueva_3: df_response[0]["resuelto"], columna_nueva_4: df_response[0]["como_resolvio"], "tokens": total_tokens_used, "seconds": response_seconds})
        except Exception as Argument:
            genai.append({'conversacion_id': fila_actual['conversacion_id'], columna_nueva_1: str(Argument), columna_nueva_2: "Error", columna_nueva_3: "Error", columna_nueva_4: "Error", "tokens": 0, "seconds": 0})

        print(f"{str(j)} = {fila_actual['conversacion_id']} = {str(df_response)}")
        j = j + 1
        if j % 30 == 0: # cada 30 registros saca un nuevo token
            token = generateTokenIBM(api_key)
    
    end_time = time.time()
    # mide tiempo de respuesta
    response_time = end_time - start_time
    
    print(f"Tiempo de respuesta: {response_time} segundos.")
    
    # agrega columna nueva al dataframe con los valores generados
    df_genai = pd.DataFrame(genai)

    return df_genai


In [112]:
# corre proceso
df_genai = proceso_genai(df_join)

0 = 64e510a1-eb7b-4b27-bd26-19f88fb4ce22 = [{'tema': 'Consulta de saldo de tarjeta', 'problema': 'Cliente desconoce saldo adeudado', 'resuelto': 'SI', 'como_resolvio': 'Confirmó monto y fecha de pago'}]
1 = c68e4cee-92bc-4868-ad78-dd8b9fa86389 = [{'tema': 'Reposición de tarjeta débito', 'problema': 'Dato de tarjeta compartido con contacto fraudulento', 'resuelto': 'SI', 'como_resolvio': 'Realizó reposición de tarjeta débito'}]
2 = 82cfc9f4-ffcb-49d2-ad9d-bb6a4fe70cb4 = [{'tema': 'Pago y cálculo de tarjeta Visa', 'problema': 'Confusión sobre moneda y pago mínimo', 'resuelto': 'SI', 'como_resolvio': 'Explicación de proceso de pago y conversion'}]
3 = 56c88f71-7a1e-49e6-9070-62a682699f53 = [{'tema': 'Cambio de titularidad en cuentas CERA', 'problema': 'Cliente necesita cotitulares en cuentas CERA', 'resuelto': 'NO', 'como_resolvio': '-'}]
4 = 078d23c6-e80f-43f6-a93f-45a920e86db8 = [{'tema': 'Diferencia en depósito de bonos', 'problema': 'Diferencia en el monto depositado por la venta de b

In [113]:
df_genai.head(10)

,conversacion_id,tema,problema,resuelto,como_resolvio,tokens,seconds
0,64e510a1-eb7b-4b27-bd26-19f88fb4ce22,Consulta de saldo de tarjeta,Cliente desconoce saldo adeudado,SI,Confirmó monto y fecha de pago,1799,4.40
1,c68e4cee-92bc-4868-ad78-dd8b9fa86389,Reposición de tarjeta débito,Dato de tarjeta compartido con contacto fraudu...,SI,Realizó reposición de tarjeta débito,2577,2.19
2,82cfc9f4-ffcb-49d2-ad9d-bb6a4fe70cb4,Pago y cálculo de tarjeta Visa,Confusión sobre moneda y pago mínimo,SI,Explicación de proceso de pago y conversion,2097,2.13
3,56c88f71-7a1e-49e6-9070-62a682699f53,Cambio de titularidad en cuentas CERA,Cliente necesita cotitulares en cuentas CERA,NO,-,1413,1.99
4,078d23c6-e80f-43f6-a93f-45a920e86db8,Diferencia en depósito de bonos,Diferencia en el monto depositado por la venta...,NO,-,1957,1.98
5,cfa82032-eb23-4b10-8c93-79655bd1b04f,Consultas sobre cuenta y tarjeta de crédito,Dificultad para gestionar cuenta paterna y sol...,NO,-,3390,2.14
6,d962f292-70ba-48ef-aae1-6ff2203d10f1,Validación de cliente,Ninguno,SI,Identificó al cliente,652,1.52
7,88f55f5c-9d99-4a09-8a5b-aefbc3322513,0,Error,Error,Error,0,0.00
8,4d6f2de7-9d88-49b5-9f74-7ebdfc522366,Consulta de saldo con descuento,No encuentra el descuento en su resumen,SI,Explicó detalladamente como calcular el saldo ...,1750,2.63
9,088da426-2540-4ece-8456-0c531a74a371,Recuperación de claves,No puede recuperar claves,SI,Generó reclamo para llamada de soporte,1290,1.92


In [114]:
df_genai.shape

(307, 7)

In [115]:
# grabar archivo
df_genai.to_csv(archivo_csv_exportar, sep=',', index=False, encoding='utf-8')

In [1]:
## Ejecutar la notebook hasta acá